# Imports

In [1]:
import numpy as np
import pandas as pd

from functools import reduce


import plotly.express as px
import plotly.graph_objects as go



import warnings
warnings.filterwarnings('ignore')

In [73]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


In [74]:
dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung']
    
    
    

## ToDo

- % Zuwachs: Either since 2015 or Benutzerdefiniert
- Waterfall graphs for %, stacked barchart for Gesamtvolum
- Top/Bottom 10
- DF to the bottom

Filter Steps:
 1. Only BMZ
 2. 100% of full, 50% of Partial Financing

# World

### Basic Processing of Data


In [75]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



,amount_2013,clim_rel_amount_2013,clim_rel_percent_2013,amount_2014,clim_rel_amount_2014,clim_rel_percent_2014,amount_2015,clim_rel_amount_2015,clim_rel_percent_2015,amount_2016,...,clim_rel_percent_2019,amount_2020,clim_rel_amount_2020,clim_rel_percent_2020,amount_2021,clim_rel_amount_2021,clim_rel_percent_2021,amount_2022,clim_rel_amount_2022,clim_rel_percent_2022
Recipient Name,,,,,,,,,,,,,,,,,,,,,
Argentina,4.394577,1.273991,0.289901,4.975084,1.569978,0.315568,48.619634,1.161736,0.023894,4.122951,...,0.187780,3.203587,0.647809,0.202214,4.035065,0.836634,0.207341,1.199731,0.187548,0.156325
Brazil,117.984860,84.179631,0.713478,60.828641,35.184794,0.578425,50.015792,27.143116,0.542691,103.262959,...,0.553333,37.305778,18.056351,0.484010,68.298989,32.506532,0.475945,42.928006,13.720415,0.319615


In [ ]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


globe_df = globe_df.loc[[142]]

globe_df

### Prepare data for stacked barchart


In [ ]:

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000



melted_df.to_csv("../upload_data/globe_df")

### Plot stacked bar

In [15]:
read_df[read_df['Year'].between(2018,2018)]

,Unnamed: 0,Type_Year,Amount,Year,Type
5,5,clim_rel_amount_2018,1.659673e+09,2018,Climate Finance
15,15,non_clim_amount_2018,6.837719e+09,2018,Other Funds


In [18]:
read_df = pd.read_csv('../upload_data/globe_df')


fig = px.bar(read_df, x='Year', y='Amount', color='Type',
            title='Global Financing Totals',
            labels={'Amount': 'Financing Amount (Euros)', 'Year': 'Year'},
            category_orders={'Type': ['Other Funds','Climate Finance']},
            color_discrete_map={'Other Funds': 'orange', 'Climate Finance': 'green'})# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

### Percentage

In [ ]:
years = [col.split('_')[-1] for col in globe_df.columns if col.startswith('amount_')]

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    clim_rel_percent_col = f'clim_rel_percent_{year}'
    
    # Calculate the clim_rel_percent column for the year
    # Note: It's a good practice to handle division by zero or check if 'amount_' is not zero to avoid ZeroDivisionError.
    globe_df[clim_rel_percent_col] = (globe_df[clim_rel_amount_col] / globe_df[amount_col]) * 100
    
    

In [ ]:
selected_columns = [col for col in globe_df.columns if col.startswith('clim_rel_percent_')]
filtered_df = globe_df[selected_columns]

In [ ]:

# Melt the DataFrame to get it in the right shape for plotting
melted_df = filtered_df.reset_index().melt(id_vars=['index'], var_name='Year', value_name='Percentage')

# Clean the 'Year' column to keep only the year part
melted_df['Year'] = melted_df['Year'].str.replace('clim_rel_percent_', '')

melted_df = melted_df.drop(columns = 'index')

In [ ]:
melted_df

In [ ]:
melted_df['Change'] = melted_df['Percentage'].diff()

In [ ]:
melted_df.to_csv('../upload_data/globe_waterfall.csv')

In [19]:
globe_waterfall_df = pd.read_csv('../upload_data/globe_waterfall.csv')


In [22]:
globe_waterfall_df[globe_waterfall_df['Year'].between(2013,2018)]

,Unnamed: 0,Year,Percentage,Change
0,0,2013,15.026220,NaN
1,1,2014,15.085999,0.059779
2,2,2015,16.924634,1.838634
3,3,2016,18.331768,1.407134
4,4,2017,19.000573,0.668805
5,5,2018,19.531556,0.530983


In [30]:
globe_waterfall_df = pd.read_csv('../upload_data/globe_waterfall.csv')

initial_value = globe_waterfall_df['Percentage'].iloc[0] - globe_waterfall_df['Change'].iloc[1] # Subtract the first actual change to get the starting point


fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = ["absolute"] + ["relative"] * (len(globe_waterfall_df) - 1), # The first measure is absolute, others are relative
    x = globe_waterfall_df['Year'].astype(str),
    textposition = "outside",
    text = globe_waterfall_df['Change'].round(2).astype(str),
    y = [initial_value] + globe_waterfall_df['Change'].tolist()[1:], # The initial value plus the changes
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = {
            'text': "Yearly Percentage Change in Climate Finance",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis = {"type":"category"},
        yaxis = {"title":"Percentage"},
)
fig.show()

### Nice DF

In [ ]:
pivot_df = df.pivot_table(index='Year', columns='Type', values='Amount', aggfunc='sum').reset_index()


In [62]:
pivot_df = read_df[['Amount', 'Year', 'Type']].pivot_table(
    index='Year', 
    columns='Type', 
    values='Amount', aggfunc='sum').reset_index()





In [67]:
full_globe_df = pivot_df.merge(globe_waterfall_df, left_on="Year", right_on="Year").set_index('Year').drop(columns='Unnamed: 0')

In [69]:
full_globe_df.to_csv('../upload_data/globe_df_to_show.csv')

In [70]:
full_globe_df = pd.read_csv('../upload_data/globe_df_to_show.csv')

In [71]:
full_globe_df

,Year,Climate Finance,Other Funds,Percentage,Change
0,2013,8.455634e+08,4.781689e+09,15.026220,NaN
1,2014,1.036792e+09,5.835755e+09,15.085999,0.059779
2,2015,9.951812e+08,4.884894e+09,16.924634,1.838634
3,2016,1.352222e+09,6.024166e+09,18.331768,1.407134
4,2017,1.457414e+09,6.212956e+09,19.000573,0.668805
5,2018,1.659673e+09,6.837719e+09,19.531556,0.530983
6,2019,2.007335e+09,7.118227e+09,21.996836,2.465280
7,2020,2.195044e+09,8.623938e+09,20.288824,-1.708012
8,2021,2.490046e+09,8.626520e+09,22.399419,2.110595
9,2022,2.830265e+09,9.238494e+09,23.451170,1.051751


# Country Sum

In [99]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)




In [100]:
df_merged.to_csv("../upload_data/all_country_df.csv")

In [102]:
all_country_df = pd.read_csv("../upload_data/all_country_df.csv")

In [106]:
countries = all_country_df['Recipient Name'].unique()

In [108]:
countries

array(['Afghanistan', 'Africa, regional', 'Albania', 'Algeria',
       'America, regional', 'Angola', 'Argentina', 'Armenia',
       'Asia, regional', 'Azerbaijan', 'Bangladesh', 'Belarus', 'Belize',
       'Benin', 'Bhutan', 'Bilateral, unspecified', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Burkina Faso',
       'Burundi', "C(te d'Ivoire", 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Caribbean & Central America, regional',
       'Central African Republic', 'Central Asia, regional', 'Chad',
       'Chile', "China (People's Republic of)", 'Colombia', 'Congo',
       'Costa Rica', 'Cuba', "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Eritrea',
       'Eswatini', 'Ethiopia', 'Europe, regional',
       'Far East Asia, regional', 'Fiji', 'Gambia', 'Georgia', 'Ghana',
       'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti',
       'Honduras',

In [87]:

df_merged = df_merged[df_merged.index.isin(['Argentina', 'Brazil'])]

In [89]:
df_merged

,amount_2013,clim_rel_amount_2013,clim_rel_percent_2013,amount_2014,clim_rel_amount_2014,clim_rel_percent_2014,amount_2015,clim_rel_amount_2015,clim_rel_percent_2015,amount_2016,...,clim_rel_percent_2019,amount_2020,clim_rel_amount_2020,clim_rel_percent_2020,amount_2021,clim_rel_amount_2021,clim_rel_percent_2021,amount_2022,clim_rel_amount_2022,clim_rel_percent_2022
Recipient Name,,,,,,,,,,,,,,,,,,,,,
Argentina,4.394577,1.273991,0.289901,4.975084,1.569978,0.315568,48.619634,1.161736,0.023894,4.122951,...,0.187780,3.203587,0.647809,0.202214,4.035065,0.836634,0.207341,1.199731,0.187548,0.156325
Brazil,117.984860,84.179631,0.713478,60.828641,35.184794,0.578425,50.015792,27.143116,0.542691,103.262959,...,0.553333,37.305778,18.056351,0.484010,68.298989,32.506532,0.475945,42.928006,13.720415,0.319615


In [94]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


globe_df = globe_df.loc[[globe_df.index[-1]]]

globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
2,122.379437,85.453621,65.803725,36.754772,98.635426,28.304852,107.38591,51.585075,111.338307,76.764691,70.240281,36.681652,61.787357,32.796035,40.509365,18.704161,72.334054,33.343165,44.127737,13.907963


In [93]:
globe_df.index[-1]

2

# Country Compare

# Country Breakdown

In [ ]:
use_df = df_dictionary['2022'][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [ ]:

use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

In [ ]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

In [ ]:
use_df['clim_rel'] = use_df.apply(determine_value, axis=1)


In [ ]:
use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']

In [ ]:
grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]

In [ ]:
grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']

In [ ]:
grouped_df

In [ ]:
processed_grouped_dfs = []

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']].reset_index()
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_processed_df = grouped_df.rename(columns={'clim_rel_percent': key})

    processed_grouped_dfs.append(grouped_processed_df[['Recipient Name',key]])

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


# df_long.to_csv('../upload_data/country_specific_df.csv')



In [ ]:
df_long.to_csv('../upload_data/newmethod_country_specific_df.csv')


In [ ]:
df_long

In [ ]:
processed_grouped_dfs = []

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']

    use_df.loc[use_df['Climate Relevance'] > 1, 'Climate Relevance'] = 1

    first_group_df = use_df.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']].reset_index()

    df_grouped = first_group_df.groupby('Recipient Name').apply(
        lambda x: (x[x['Climate Relevance'] == 1]['Amount'].sum()\
                   / x['Amount'].sum()) * 100).reset_index(name='Climate Relevance %')

    grouped_processed_df = df_grouped.rename(columns={'Climate Relevance %': key})

    processed_grouped_dfs.append(grouped_processed_df)

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


df_long.to_csv('../upload_data/country_specific_df.csv')



In [ ]:
df_long

In [ ]:
df_long = pd.read_csv('../upload_data/country_specific_df.csv')[['Recipient Name', 'Year', 'Value']]

In [ ]:
df_long

In [ ]:
countries = df_long['Recipient Name'].unique()
# countries

In [ ]:
selected_df = df_long[df_long['Recipient Name'].isin(['India', 'Brazil', 'Ukraine', 'Namibia', 'South Africa'])]

In [ ]:
fig = px.line(selected_df, 
              x='Year', 
              y='Value', 
              color='Recipient Name',
              title='Value by Year for Selected Country',
              labels={'Value': 'Climate Relevant Financing (%)', 'Year': 'Year'},
              markers=True)

fig.show()

# Globe Graph

In [ ]:
def df_compiler(dictionary_of_df):
    
    processed_grouped_dfs = []
    
    for key in dictionary_of_df.keys():
        
        use_df = dictionary_of_df[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
        
        use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
        use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
        
        use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']
        
        use_df.loc[use_df['Climate Relevance'] > 2, 'Climate Relevance'] = 2
        
        use_df['Amount'] = use_df['Amount'] * 1_000_000
        
        grouped_processed_df = use_df.groupby(['Climate Relevance']).sum()[['Amount']]
        
        grouped_processed_df = grouped_processed_df.rename(columns={'Amount': key})
        
        processed_grouped_dfs.append(grouped_processed_df)
        
    merged_df = pd.concat(processed_grouped_dfs, axis=1)
    
    sum_row = merged_df.sum()
    sum_df = pd.DataFrame([sum_row], index=['Total Financing'])
    
    merged_df = merged_df.append(sum_df)

    merged_df.index = ['No Climate Relevance','Partial Climate Relevance',
                       'High Climate Relevance', 'Total Financing']

    return merged_df


merged_df = df_compiler(df_dictionary)

merged_df.to_csv('../upload_data/global_df.csv')

merged_df

In [ ]:
globe_df = pd.read_csv('../upload_data/global_df.csv')
globe_df

In [ ]:

df_long = merged_df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']



fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Global BMZ Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:
df_long = df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']

# Filter out the 'Total Financing' row to avoid double counting in the stacked bar
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']

# Create the stacked bar chart
fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Total Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:


testing_2022 = df_2022[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [ ]:
testing_2022['Climate Adaptation'] = testing_2022['Climate Adaptation'].fillna(0)
testing_2022['Climate Mitigation'] = testing_2022['Climate Mitigation'].fillna(0)

In [ ]:
testing_2022['Climate Relevance'] = testing_2022['Climate Adaptation'] + testing_2022['Climate Mitigation']

In [ ]:
testing_2022.loc[testing_2022['Climate Relevance'] > 2, 'Climate Relevance'] = 2


In [ ]:
testing_2022.groupby(['Climate Relevance']).sum()[['Amount']].rename(columns={'Amount': '2022'})

In [ ]:
grouped_test = testing_2022.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']]